In [1]:
import pandas as pd
import os
import numpy as np
import math

In [3]:
def replace_feature_names(feature_name):
    
    feature_name = feature_name.replace(' ','_')
    feature_name = feature_name.replace('-','_')
    feature_name = feature_name.replace("'",'prime')
    feature_name = feature_name.replace("*",'')
    feature_name = feature_name.replace(",",'_')
    
    return feature_name

def make_fs_dataset(feature_size_list, cutoff_type, m_full_data_df, rwr_result_df, output_dir, ext_data_file, sep_method):
    
    for feature_size in feature_size_list: 
        print (feature_size)

        if cutoff_type == "top":
            cutoff_dir = "%s/top%s" % (output_dir, feature_size)
            feature_list = list(rwr_result_df["acpa"][:feature_size+1])
            
        if cutoff_type == "percentage":
            cutoff_dir = "%s/%s" % (output_dir, feature_size)
            feature_list = list(rwr_result_df["acpa"])
            
            threshold = feature_size.split('p')[0]
            threshold = float(threshold)
            threshold = len(feature_list) * threshold * 0.01
            threshold = int(math.ceil(threshold)) #round number then change to int
 
            feature_list = list(rwr_result_df["acpa"][:threshold])
            # print (len(feature_list), feature_list)
        
        training_file = "%s/full_metabolomics.fs.train.tsv" % cutoff_dir

        if os.path.isdir(cutoff_dir) == False:
            os.system('mkdir %s' % (cutoff_dir))

        feature_list.insert(0, "acpa")
    
        #Prepare test dataset: CASE1
        if sep_method == "tab":
            m_test_data_df = pd.read_csv('%s' % ext_data_file, sep="\t", index_col = 0)
        else:
            m_test_data_df = pd.read_csv('%s' % ext_data_file, sep=",", index_col = 0)
        m_test_data_df.index = m_test_data_df.index.map(replace_feature_names)
        
        full_feature_list = list(m_full_data_df.index.values)
        test_feature_list = list(m_test_data_df.index.values)
        common_feature_list = list(set(feature_list) & set(test_feature_list) & set(full_feature_list))
        
        if "acpa" in common_feature_list:
            common_feature_list.remove("acpa")
        common_feature_list = sorted(common_feature_list)
        common_feature_list.insert(0, "acpa")

        if feature_size == "100p":
            print ("MAX feature size:",len(common_feature_list))
    
     
        m_test_data_df = m_test_data_df.loc[common_feature_list]
        test_file = "%s/full_metabolomics.fs.test.scaled.tsv" % cutoff_dir
        m_test_data_df.to_csv(test_file, sep="\t")         
       
        #Prepare training dataset
        fs_m_full_data_df = m_full_data_df.loc[common_feature_list]
        fs_m_full_data_df.to_csv(training_file, sep="\t")


In [4]:
#extract feature list from RWR results
rwr_result_file = "../../../analysis/full_data_metabolomics_neg_pos/post_network_enet/metabolomics_full_data.RWR.result.txt"

rwr_result_df = pd.read_csv(rwr_result_file, sep="\t")

In [5]:
#prepare dataset from full data: metabolomics
m_full_data_file = "../../../analysis/full_data_metabolomics_neg_pos/metabolomics_ML_ready.v2.tsv"

m_full_data_df = pd.read_csv(m_full_data_file, sep="\t", index_col=0)
m_full_data_df.index = m_full_data_df.index.map(replace_feature_names)

In [6]:
#prepare test dataset from plasma metabolomics
output_dir = "../../../analysis_revision/external_validation/ra_art_metabolomics/neg_vs_pos"
ext_data_file = "../../../analysis_revision/external_validation/ra_art_metabolomics/data/RA_art_metabolomics.scaled.patient_info.tsv"

feature_size_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
make_fs_dataset(feature_size_list, "top", m_full_data_df, rwr_result_df, output_dir, ext_data_file, "tab")

feature_size_list = ["1p", "5p", "10p","20p","25p", "30p","40p","50p","100p"]
make_fs_dataset(feature_size_list, "percentage", m_full_data_df, rwr_result_df, output_dir, ext_data_file, "tab")

10
20
30
40
50
60
70
80
90
100
200
300
400
500
600
700
800
900
1000
1p
5p
10p
20p
25p
30p
40p
50p
100p
MAX feature size: 554
